# Graph that calculates the footprint

Let's start by having all agents share the same state. We're going to have the following nodes

* Planner
* Manufacturing
* Raw materials
* Packaging
* Transportation
* Use phase
* End of life


In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
from time import sleep
import os

while 'requirements.txt' not in os.listdir():
    os.chdir('..')

# Load environment variables from .env.local file
load_dotenv(dotenv_path='.env.local')

# Update with your name to group your own traces
os.environ['LANGCHAIN_PROJECT'] = 'steve-fap-sandbox'

In [2]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import Field, BaseModel
from langchain_openai import ChatOpenAI
from state import FootprintState

# Tools

## Page Analyzer

In [3]:
from langchain_core.tools import tool

def page_analyzer(url: str, questions: list[str]) -> list:
    # If the url has not been crawled, crawl and cache it
    # {"brand": "Apple", "category": "cellphone", "description": "An iPhone 15"}
    answers = []
    for question in questions:
        if "packaging" in question:
            answers.append({"question":question, "answer":"A cardboard box"})
        elif "brand" in question:
            answers.append({"question":question, "answer":"Apple"})
        elif "category" in question:
            answers.append({"question":question, "answer":"cellphone"})
        elif "description" in question:
            answers.append({"question":question, "answer":"An Apple iPhone 15 Pro"})
        elif "weight" in question:
            answers.append({"question":question, "answer":0.17})
        else:
            answers.append({"question":question, "answer":f"not found"})
    return answers

@tool
def page_analyzer_tool(url: str, questions: list[str]) -> list:
    """Given a list of questions about a product described at url, returns a
    list of question and answer pairs. If an answer for a question cannot be
    found, it will be "not found"."""
    return page_analyzer(url, questions)


## Material Estimator

In [4]:
def material_estimator(url: str) -> float:
    return [{
        "kgCO2e": 100,
        "description": f"This is a test description for {url}",
        "source": "This is a test source"
    }]

@tool
def material_estimator_tool(url) -> float:
    """For a given product url, returns a list of materials in the product."""
    return material_estimator(url)

#material_estimator("6.12 inch LCD display for a cell phone", "manufacturing")

# Agents

## Planner

In [5]:
from langchain_openai.chat_models import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# The planner orchestrates the work
def planner(state: FootprintState):
    print('Running Planner')

    starting_questions = [
        "What is the brand of the product?",
        "What is the product category?",
        "What is a 1 sentence, factual description of the product?",
        "What is the product's weight?"
    ]
    
    # Call the page analyzer directly to get the high level product details
    high_level_product_details = page_analyzer(state["url"], starting_questions)
    print(high_level_product_details)
    materials = material_estimator(state["url"])
    print(materials)
    
    return {
        "messages": [{"role": "ai", "content": "planner"}],
        "brand": high_level_product_details[0]["answer"],
        "category": high_level_product_details[1]["answer"],
        "description": high_level_product_details[2]["answer"],
    }

planner({"url": "https://www.apple.com/iphone-15-pro/"})

Running Planner
[{'question': 'What is the brand of the product?', 'answer': 'Apple'}, {'question': 'What is the product category?', 'answer': 'cellphone'}, {'question': 'What is a 1 sentence, factual description of the product?', 'answer': 'An Apple iPhone 15 Pro'}, {'question': "What is the product's weight?", 'answer': 0.17}]
[{'kgCO2e': 100, 'description': 'This is a test description for https://www.apple.com/iphone-15-pro/', 'source': 'This is a test source'}]


{'messages': [{'role': 'ai', 'content': 'planner'}],
 'brand': 'Apple',
 'category': 'cellphone',
 'description': 'An Apple iPhone 15 Pro'}

## Manufacturing

In [6]:
# The maufacturing agent estimates the footprint of assembling the product
def manufacturing_phase(state: FootprintState):
    print('Running raw materials and manufacturing')
    sleep(4)
    print('Finished raw materials and manufacturing')
    return {"messages": [{"role": "ai", "content": "manufacturing"}]}

## Summarizer

In [7]:
def summarizer(state: FootprintState):
    print('Running summarizer')
    sleep(3)
    print('Finished summarizer')
    return {"messages": [{"role": "ai", "content": "summarizer"}]}


# Assemble the Analysis Graph

In [8]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver
from agents.packaging import packaging_phase
from agents.transportation import transportation_phase
from agents.use import use_phase
from agents.eol import eol_phase

# See https://langchain-ai.github.io/langgraph/how-tos/persistence_postgres/
DB_URI = os.environ['DB_DEV_CONNECTION']
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

model = ChatOpenAI(model_name="o3", temperature=0)

# Setup the building with that state
graph_builder = StateGraph(FootprintState)

graph_builder.add_node("planner", planner)
graph_builder.add_edge(START, "planner")
graph_builder.add_node("manufacturing_phase", manufacturing_phase)
graph_builder.add_node("packaging_phase", packaging_phase)
graph_builder.add_node("transportation_phase", transportation_phase)
graph_builder.add_node("use_phase", use_phase)
graph_builder.add_node("eol_phase", eol_phase)
graph_builder.add_node("summarizer", summarizer)


phases = ["manufacturing_phase", "packaging_phase", "transportation_phase", "use_phase", "eol_phase"]
for phase in phases:
    graph_builder.add_edge("planner", phase)

# By using a list for the first argument, the summarizer knows to wait for all
# phases to finish before running
graph_builder.add_edge(phases, "summarizer")
graph_builder.add_edge("summarizer", END)

# Compile with no checkpointer just to generate the graph image
# Mermaid is having issues...
#from IPython.display import Image
#Image(graph_builder.compile().get_graph().draw_mermaid_png())
graph_builder.compile().get_graph().print_ascii()

                                                              +-----------+                                                              
                                                              | __start__ |                                                              
                                                              +-----------+                                                              
                                                                    *                                                                    
                                                                    *                                                                    
                                                                    *                                                                    
                                                          ****+---------+******                                                          
                                  

In [ ]:
config = {"configurable": {"thread_id": "10"}}
with ConnectionPool(
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)

    #graph = graph_builder.compile(checkpointer=checkpointer)
    # For testing, start over each time and don't use a checkpointer
    graph = graph_builder.compile()
    # res = graph.invoke({
    #     "user_input": "calculate the cradle to gate footprint",
    #     "url": "https://www.apple.com/iphone-15-pro/",
    #     "messages": [("human", "This is a test message")]
    # }, config)
    # checkpoint = checkpointer.get(config)

    events = graph.stream(
        {
            "user_input": "calculate the cradle to gate footprint",
            "url": "https://www.apple.com/iphone-15-pro/",
            "messages": [("human", "This is a test message")]
        },
        config, 
        # See https://langchain-ai.github.io/langgraph/how-tos/streaming/
        stream_mode=["updates", "values"]
    )
    # heads up, some print statements are actually debug prints from the tools and agents
    for mode, event in events:
        # This will stream the LLM calls as tokens are generated (suppressing here, gets messy)
        # if mode == "messages":
        #     print("MESSAGES:")
        #     msg, metadata = event
        #     if msg.content:
        #         print(msg.content, end="", flush=True)
        if mode == "updates":
            print("UPDATE:", event)

        elif mode == "values":
            print("VALUES:", event)

#display(res)
# print(checkpoint)

VALUES: {'messages': [HumanMessage(content='This is a test message', additional_kwargs={}, response_metadata={}, id='cf486573-2c66-437d-aa35-242705072784')], 'user_input': 'calculate the cradle to gate footprint', 'url': 'https://www.apple.com/iphone-15-pro/'}
Running Planner
[{'question': 'What is the brand of the product?', 'answer': 'Apple'}, {'question': 'What is the product category?', 'answer': 'cellphone'}, {'question': 'What is a 1 sentence, factual description of the product?', 'answer': 'An Apple iPhone 15 Pro'}, {'question': "What is the product's weight?", 'answer': 0.17}]
[{'kgCO2e': 100, 'description': 'This is a test description for https://www.apple.com/iphone-15-pro/', 'source': 'This is a test source'}]
UPDATE: {'planner': {'messages': [{'role': 'ai', 'content': 'planner'}], 'brand': 'Apple', 'category': 'cellphone', 'description': 'An Apple iPhone 15 Pro'}}
VALUES: {'messages': [HumanMessage(content='This is a test message', additional_kwargs={}, response_metadata={}

# Testing

In [23]:
from agents.transportation import transportation_phase
from agents.use import use_phase
from agents.eol import eol_phase
#transportation_phase({"brand": "Apple", "category": "cellphone", "description": "An iPhone 15"})
#use_phase({"brand": "Apple", "category": "cellphone", "description": "An iPhone 15"})
eol_phase({"brand": "Apple", "category": "cellphone", "description": "An iPhone 15"})

TOOL: Emissions Factor Finder recycling of cellphone end-of-life
TOOL: Emissions Factor Finder disposal of cellphone end-of-life
TOOL: Emissions Factor Finder transportation of cellphone end-of-life
kgCO2e=0.5 units='per cellphone' description='The carbon emissions factor for the disposal of a cellphone at its end-of-life phase, considering typical recycling and waste management processes.'
kgCO2e=0.5 units='per kg' description='The carbon emissions factor for the transportation of a cellphone during its end-of-life phase, which includes the collection and transportation to recycling or disposal facilities.'
kgCO2e=0.5 units='per cellphone' description='The carbon emissions factor for recycling a cellphone at its end-of-life phase. This estimate considers the energy and resources required to disassemble, process, and recycle the materials in a typical cellphone.'


{'eol': {'carbon': 1.1,
  'summary': 'The end-of-life phase of an iPhone 15 involves recycling, disposal, and transportation, resulting in a carbon footprint of approximately 1.1 kg CO2e. This estimate considers the emissions from processing, waste management, and logistics.',
  'messages': [HumanMessage(content='Brand: Apple\nCategory: cellphone\nDescription: An iPhone 15', additional_kwargs={}, response_metadata={}, id='bc5f7634-b64b-40ad-a6c4-70f452e4ee78'),
   AIMessage(content="To estimate the carbon footprint of the end-of-life phase for an iPhone 15, we need to consider the processes involved in this phase, such as recycling, disposal, and transportation. Let's start by finding the emissions factors for these processes. \n\nI'll look for emissions factors for the following processes related to the end-of-life phase of a cellphone:\n\n1. Recycling\n2. Disposal\n3. Transportation\n\nLet's proceed with finding these emissions factors.", additional_kwargs={'tool_calls': [{'id': 'cal